### **4.3 - Backfill and Catchup**

**¿Qué es un DAG Run?**

Un DAG con una fecha_de_inicio, un intervalo_de_programación y posiblemente una fecha_de_finalización define una serie de intervalos que el programador convierte en DagRuns individuales para ejecutar. Así, cada vez que se activa un DAG, se crea un DAGRun que es ejecutado por el Programador. Tenga en cuenta que un DAGRun también puede ser creado utilizando el comando airflow trigger_dag a través de la interfaz de línea de comandos. Un DAGRun es atómico, lo que significa que puede funcionar por sí mismo y no necesita compartir recursos con ningún otro DAGRun. También es idempotente, lo que significa que puedes reiniciar el mismo DAGRuns una y otra vez, siempre producirá los mismos resultados. Esto es extremadamente importante. Si ejecutas tu DAG dos veces, en la misma fecha, y no obtienes el mismo resultado, entonces puedes estar en problemas. La idempotencia debe ser aplicada a tus tareas para evitar obtener errores si necesitas reiniciarlas por cualquier razón. Aquí pongo un pequeño esquema de cómo es un DAGRuns. 

<center><img src="https://i.postimg.cc/8PwbTmnx/a629.png"></center>

**`Catchup process`**

Digamos que tiene el siguiente DAG que comienza el 1 de enero de 2019 a medianoche, programado (scheduled) para ejecutarse todos los días a medianoche. Estos valores corresponden a lo que puedes ver aquí. En el momento actual, todo funciona bien, tu DAG se dispara (gets triggered) como se esperaba el 2 de enero de 2019 a medianoche y el DAGRun representado por el cuadrado ha terminado con éxito. Fíjate en la execution_date y la start_date que se muestran a continuación.

<center><img src="https://i.postimg.cc/XqYpLdcy/a630.png"></center>

Ahora, digamos que por la razón que sea, necesitas dejar de programar tu DAG. Al día siguiente día, tu DAG se pone en pausa y el DAGRun correspondiente a la ejecución del 2 de enero de 2019 a medianoche no se ejecuta el 3 de enero de 2019 a medianoche como debería ser normalmente. Al día siguiente, su DAG sigue en pausa. De nuevo, el DAGRun correspondiente a la execution_date el 3 de enero de 2019 a medianoche, no se ejecuta en la fecha actual. A partir de ese momento, tenemos 2 DAGRuns en retraso.

<center><img src="https://i.postimg.cc/wM43GM04/a631.png"></center>

Finalmente, después de dos días, se decide volver a activar el DAG para programarlo (scheduling) de nuevo. El Scheduler comprueba el estado de los diferentes DAGRuns y descubre que algunos de ellos no se han ejecutado como se esperaba. Por lo tanto, los dos DAGRuns retrasados se ejecutan de forma secuencial o concurrente dependiendo del executor utilizado.

<center><img src="https://i.postimg.cc/dVyD5qdn/a632.png"></center>

Una vez que los DAGRuns han terminado de ejecutarse, mantienen el intervalo de 1 día para su execution_date pero todos terminan con la misma start_date como puedes ver aquí ya que se han ejecutado al mismo tiempo. Esa es la única razón por la que se puede tener una diferencia mayor que el schedule_interval entre la start_date y la execution_date de tu DAGRun. Por cierto, todo este proceso ocurre sólo si el parámetro catchup está establecido en True. Ese es el caso por defecto.

<center><img src="https://i.postimg.cc/Xv6qhTkW/a633.png"></center>

Hay dos maneras de configurar el parámetro catchup. O bien decides establecerlo localmente en la instanciación de tu DAG. O bien decides cambiar el valor por defecto desde el archivo de configuración **`airflow.cfg`** en el parámetro llamado **`catchup_by_default`**, que por defecto, es igual a True, ***`catchup_by_default = True`***.

<center><img src="https://i.postimg.cc/BZm86745/a634.png"></center>